# Study of feature importance

In this notebook we apply several methods to understand which features are important for the predictions

In [ ]:
# ToDo: comments to be completed

In [ ]:
import umap
import umap.plot

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import pickle
import pathlib as pl
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import f1_score, average_precision_score, precision_score, recall_score, jaccard_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from sklearn.metrics import entropy

import lightgbm as lgb
import shap

In [ ]:
def load_pckl(file_name, path=None):
    if path is not None:
        file_name = os.path.join(path, file_name)

    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    return data


def save_pckl(d, file_name, pr=None, path=None):
    if path is not None:
        file_name = os.path.join(path, file_name)

    with open(file_name, 'wb') as f:
        pickle.dump(d, f, protocol=pr if pr is not None else pickle.DEFAULT_PROTOCOL)

In [ ]:
data_root = pl.Path(r'D:\Projects_MV\conv_paint\data')

In [ ]:
datasets = [ 'actin', 'fret', 'nuclei', 'spindle', 'worms', 'worms2']
ds = datasets[0]

In [ ]:
blocks = [1, 64, 64, 256, 512, 512, 64, 64, 256, 512, 512, 1, 64, 64, 256, 512, 512, 64, 64, 256, 512, 512]

In [ ]:
sum(blocks)

In [ ]:
csb = np.cumsum([0]+blocks)
csb = list(csb)

In [ ]:
for ds in datasets:
    p = data_root/'features'/ds
    for pi in p.glob('embedding_sample_*.pckl'):
        print(pi)

        d_emb = load_pckl(pi)

In [ ]:
d_emb.keys()

In [ ]:
d_emb['f'].shape, d_emb['l'].shape

In [ ]:
plt.hist( d_emb['l']);

# Model fitting

+try this and rf

+add tra-val split

re-eval same on mode data

In [ ]:
def eval_model(clf, X_val, y_val):
    # Predict on the validation set
    y_pred = clf.predict(X_val)
    y_pred_proba = clf.predict_proba(X_val)[:, 1:2]  # 
    
    # Calculate F1 score
    f1 = f1_score(y_val, y_pred, average='weighted')
    
    # Calculate Average Precision
    average_precision = average_precision_score(y_val==2, y_pred_proba)
    
    # Calculate Precision
    precision = precision_score(y_val, y_pred, average='weighted')
    
    # Calculate Recall
    recall = recall_score(y_val, y_pred, average='weighted')
    
    # Calculate Jaccard score (intersection over union)
    jaccard = jaccard_score(y_val, y_pred, average='weighted')
    cm = confusion_matrix(y_val, y_pred, labels=clf.classes_)
        
    d = {
        'f1': f1,
        'ap': average_precision,
        'p': precision,
        'r': recall,
        'IoU': jaccard,
        'cm': cm
    }
    print('Classification Report:')
    print(classification_report(y_val, y_pred))


    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=clf.classes_)
    fig, ax = plt.subplots(figsize=(3, 3))
    disp.plot(ax=ax)
    plt.show()
    plt.close()
    return d

In [ ]:
n_per_cl = 10000
n_per_cl_sh = 1000

for ds in datasets:
    p = data_root/'features'/ds
    p = data_root/'features'/ds
    
    all_features_all_samples = []
    all_targets_all_samples = []

    for pi in p.glob('features_*.pckl'):
        if '_sample_' in str(pi):
            continue
        # print(pi)
        #try:
        d = load_pckl(pi)
        #except Exception as e:
        #    continue
        # print(d['features'][0].shape)
        features_all_samples, targets_all_samples, feature_info = d['features'][0], d['targets'][0], d['feature_info']
        all_features_all_samples.append(np.asarray(features_all_samples))
        all_targets_all_samples.append(np.asarray(targets_all_samples))

    
    all_features_all_samples = np.concatenate(all_features_all_samples, axis=0)
    all_targets_all_samples = np.concatenate(all_targets_all_samples, axis=0)
    print(f'all_features_all_samples.shape={all_features_all_samples.shape}; all_targets_all_samples.shape={all_targets_all_samples.shape}')

    def subsample_idx(class_lbl, n_per_class):
        class_indices = {}
        for class_id in set(class_lbl):
            indices = np.where(np.array(class_lbl) == class_id)[0]
            np.random.shuffle(indices)  # Shuffle the indices
            class_indices[class_id] = indices[:n_per_class]  # Take the first n_per_cl indices
        
        # Concatenate the selected indices
        selected_indices = np.concatenate(list(class_indices.values()))

        return selected_indices

    selected_indices = subsample_idx(all_targets_all_samples, n_per_cl)

    X, y = all_features_all_samples[selected_indices].copy(), all_targets_all_samples[selected_indices].copy()
    
    X_train_all, X_val_all, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=108)

    selected_indices_sh = subsample_idx(y_val, n_per_cl_sh)
    X_val_sh_all = X_val_all[selected_indices_sh].copy()
    y_val_sh = y_val[selected_indices_sh].copy()

    d_feat = {
        'X_train_all' : X_train_all,
        'X_val_all' : X_val_all,
        'X_val_sh_all' : X_val_sh_all,
        'y_train' : y_train,
        'y_val' : y_val,
        'y_val_sh' : y_val_sh,
    }
    for b, e in zip(csb[0:-1]+[csb[0]], csb[1:]+[csb[-1]]):  # last = all channels together
        X_train = X_train_all[:, b:e].copy()
        X_val = X_val_all[:, b:e].copy()
        X_val_sh = X_val_sh_all[:, b:e].copy()

        print(ds, b, e, X_train.shape, X_val.shape)
        
        clf_rf = RandomForestClassifier(n_estimators=100, n_jobs=8)
        clf_rf.fit(X=X_train, y=y_train)

        d_feat[f'eval_rf_{b}_{e}'] = eval_model(clf_rf, X_val, y_val)

        print('done rf')
    
        # We create the LGBMClassifier model and train it
        clf_km = lgb.LGBMClassifier(colsample_by_tree=0.8, n_jobs=8)
        clf_km.fit(X=X_train, y=y_train)

        d_feat[f'eval_km_{b}_{e}'] = eval_model(clf_km, X_val, y_val)
    
        print('done lgbm')
        explainer_rf = shap.TreeExplainer(clf_rf)
        shap_values_rf = explainer_rf.shap_values(X_val_sh, y_val_sh, check_additivity=False)
        d_feat[f'shap_rf_{b}_{e}'] = shap_values_rf

        fib = feature_info[b]
        ttl = fib[1].replace(f'_{fib[0]}', '')
        
        shap.summary_plot(shap_values_rf, plot_type="bar", plot_size=(15, 10), max_display=50, show=False)
        plt.title(ttl)
        
        plt.savefig(p/f'plots_shap_rf_{b}_{e}')
        plt.show()
        plt.close()

        print('done rf shap')
        
        explainer_km = shap.TreeExplainer(clf_km)
        shap_values_km = explainer_km.shap_values(X_val_sh, y_val_sh, check_additivity=False)
        d_feat[f'shap_km_{b}_{e}'] = shap_values_km

        shap.summary_plot(shap_values_km, plot_type="bar", plot_size=(15, 10), max_display=50, show=False)
        plt.title(ttl)
        
        plt.savefig(p/f'plots_shap_km_{b}_{e}')
        plt.show()
        plt.close()

        print('done lgbm shap')
    
    save_pckl(d_feat, f'features_sample_{n_per_cl//1000}k.pckl', path=p.as_posix())

# Entropy

In [ ]:
for ds in datasets:
    p = data_root/'features'/ds
    
    for pi in p.glob('features_sample_*k.pckl'):
        #print(pi)
        d = load_pckl(pi)
        stop
    

In [ ]:
d.keys()

In [ ]:
x_all = d['X_train_all']
y_all = d['y_train']

x_all.shape, y_all.shape

In [ ]:
ls = [1, 2, 3]
masks = {l: y_all==l for l in ls}

In [ ]:
#std difference:

x_std = {l: x_all[masks[l]].std(axis=0) for l in ls}

In [ ]:
signal_order = np.argsort(x_std[2])

In [ ]:
#plt.plot(x_std[2][signal_order])

In [ ]:
sum(x_std[2]==0)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 6))
ax[0][0].plot(x_std[2]/x_std[1])
ax[0][1].plot(x_std[2]/x_std[3])
ax[1][0].semilogy(x_std[2]/x_std[1])
ax[1][1].semilogy(x_std[2]/x_std[3])

ax[0][0].set_title('signal/bg std ratio')
ax[0][1].set_title('signal/edge std ratio')
ax[0][0].set_ylim(0, 500)
ax[0][1].set_ylim(0, 500);


# Eval subsetted performance

In [ ]:
d['eval_rf_0_1'], d['eval_rf_1_65'], d['eval_rf_2818_2882'], d['eval_rf_0_5634']

In [ ]:
column_selection = {
    ds:{n_best:{'strategy_name': [], 'column_idxs': [], 'strategy_desc':[]} for n_best in [20, 64, 128]} \
    for ds in datasets
}

## Select columns idx by highest std ratio

select best by the ratio of feature standard deviation:

In [ ]:
for ds in column_selection:
    p = data_root/'features'/ds
    
    for pi in p.glob('features_sample_*k.pckl'):
        #print(pi)
        d = load_pckl(pi)
        break # should be just one anyway
    column_selection_ds = column_selection[ds]

    x_all = d['X_train_all']
    y_all = d['y_train']
    
    ls = [1, 2, 3]
    masks = {l: y_all==l for l in ls}

    #std difference:
    
    x_std = {l: x_all[masks[l]].std(axis=0) for l in ls}
    
    s_to_b_std = x_std[2]/x_std[1]
    s_to_e_std = x_std[2]/x_std[3]

    s_to_b_std[x_std[1]==0] = 0
    s_to_e_std[x_std[3]==0] = 0

    # 1. find largest in each group
    sorting_idx_s_to_b_std = np.argsort(s_to_b_std)[::-1]
    sorting_idx_s_to_e_std = np.argsort(s_to_e_std)[::-1]
    
    for n, column_selection_ds_n in column_selection_ds.items():
        names = [f'std_ratio_sig_to_bg', f'std_ratio_sig_to_edge']
        descs = ['largest ratio of channel values std in signal region to std in background region', 
                 'largest ratio of channel values std in signal region to std in edge region']
        idxs_arrs = [sorting_idx_s_to_b_std, sorting_idx_s_to_e_std]
        for name, desc, idxs in zip(names, descs, idxs_arrs):
            sel_idxs = idxs[:n]
            column_selection_ds_n['strategy_name'].append(name)
            column_selection_ds_n['column_idxs'].append(sel_idxs)
            column_selection_ds_n['strategy_desc'].append(desc)

## Select columns idx by highest Shaps

select best by the ratio of feature standard deviation:

In [ ]:
aggregated_shaps = {}

for ds in column_selection:
    p = data_root/'features'/ds
    
    for pi in p.glob('features_sample_*k.pckl'):
        #print(pi)
        d = load_pckl(pi)
        break # should be just one anyway
    column_selection_ds = column_selection[ds]

    for k in d:
        prfx = 'shap_rf_'
        if prfx not in k:
            continue

        start_idx, end_idx = [int(v) for v in k.replace(prfx, '').split('_')]
        all_shaps = np.array(d[k])

        if k not in aggregated_shaps:
            aggregated_shaps[k] = []  # order will be DS_CLASS_SAMPLE_CHANNEL, to be concatenated by sample
        aggregated_shaps[k].append(all_shaps)
        
        per_idx_mean_shap = np.abs(all_shaps).sum(axis=0).mean(axis=0)
        
        sorting_block_idx_shaps = np.argsort(per_idx_mean_shap)[::-1]
        sorting_idx_shaps = sorting_block_idx_shaps + start_idx

        for n, column_selection_ds_n in column_selection_ds.items():
            name = f'shaps_{start_idx}_{end_idx}'
            desc = f'largest shaps in block {start_idx} to {end_idx}'
            sel_idxs = sorting_idx_shaps[:n]
            
            column_selection_ds_n['strategy_name'].append(name)
            column_selection_ds_n['column_idxs'].append(sel_idxs)
            column_selection_ds_n['strategy_desc'].append(desc)

## Select columns idx by highest Shaps averaged by all datasets

select best by the ratio of feature standard deviation:

In [ ]:
for k in aggregated_shaps:
    prfx = 'shap_rf_'
    if prfx not in k:
        continue

    start_idx, end_idx = [int(v) for v in k.replace(prfx, '').split('_')]
    all_shaps = aggregated_shaps[k]  # axes order will be DS_CLASS_SAMPLE_CHANNEL, to be concatenated by sample
    all_shaps = np.concatenate(all_shaps, axis=1)
    
    per_idx_mean_shap = np.abs(all_shaps).sum(axis=0).mean(axis=0)
    
    sorting_block_idx_shaps = np.argsort(per_idx_mean_shap)[::-1]
    sorting_idx_shaps = sorting_block_idx_shaps + start_idx

    for ds in column_selection:
        column_selection_ds = column_selection[ds]
    
        for n, column_selection_ds_n in column_selection_ds.items():
            name = f'agg_shaps_{start_idx}_{end_idx}'
            desc = f'largest aggregated over datasets shaps in block {start_idx} to {end_idx}'
            
            sel_idxs = sorting_idx_shaps[:n]
            
            column_selection_ds_n['strategy_name'].append(name)
            column_selection_ds_n['column_idxs'].append(sel_idxs)
            column_selection_ds_n['strategy_desc'].append(desc)

## Select random columns idx from all or first layer + input

In [ ]:
blocks

In [ ]:
block_first_all_raw_idx = [0, 1, 6, 12, 17]
block_first_raw_idx = [0, 1, 12]


In [ ]:
 csb  # cumsum

In [ ]:
first_and_raw_idx = []
begins = [csb[:-1][i] for i in block_first_raw_idx]
ends =   [csb[1:][i] for i in block_first_raw_idx]
for b, e in zip(begins, ends):
    first_and_raw_idx.extend(list(range(b, e)))

first_all_and_raw_idx = []
begins = [csb[:-1][i] for i in block_first_all_raw_idx]
ends =   [csb[1:][i] for i in block_first_all_raw_idx]
for b, e in zip(begins, ends):
    first_all_and_raw_idx.extend(list(range(b, e)))




In [ ]:
len(first_and_raw_idx), len(first_all_and_raw_idx)

In [ ]:
descs = ['raw+random form 1st layer', 'raw+random form 1st layer all scales']
names = ['raw_rnd_l1', 'raw_rnd_all_l1']
idxs = [first_and_raw_idx, first_all_and_raw_idx]
for desc, name, idx in zip(descs, names, idxs):
    ord_idx = [idx[0]]
    rnd_src_idx = idx[1:]
    rnd_idx = list(np.random.permutation(rnd_src_idx))
    ord_idx.extend(rnd_idx)
    
    for ds in column_selection:
        column_selection_ds = column_selection[ds]
    
        for n, column_selection_ds_n in column_selection_ds.items():
            sel_idxs = ord_idx[:n]
            
            column_selection_ds_n['strategy_name'].append(name)
            column_selection_ds_n['column_idxs'].append(sel_idxs)
            column_selection_ds_n['strategy_desc'].append(desc)

# Save selected idx_dict

In [ ]:
save_pckl(column_selection, data_root/'features'/'selected_idx.pckl')

In [ ]:
#column_selection

# Evaluation of subsetting strategies

In [ ]:
column_selection = load_pckl(data_root/'features'/'selected_idx.pckl')

In [ ]:
def call_back_per_ds(x_tra, y_tra, x_val, y_val, cond_desc, cond_name):
    print(cond_name)
    # should return metrics dict
    clf_rf = RandomForestClassifier(n_estimators=100, n_jobs=8)
    clf_rf.fit(X=x_tra, y=y_tra)

    m = eval_model(clf_rf, x_val, y_val)
    return m

In [ ]:
for ds, column_selection_ds in column_selection.items():
    p = data_root/'features'/ds
    
    for pi in p.glob('features_sample_*k.pckl'):
        #print(pi)
        d = load_pckl(pi)
        break # should be just one anyway

    x_tra_all = d['X_train_all']
    x_val_all = d['X_val_all']
    y_tra = d['y_train']
    y_val = d['y_val']

    std = x_tra_all.std(axis=0)
    x_tra_v = x_tra_all[:, std>0]
    x_val_v = x_val_all[:, std>0]
    
    scaler = StandardScaler()
    x_tra_vn = scaler.fit_transform(x_tra_v)
    x_val_vn = scaler.transform(x_val_v)
    
    # print(x_tra_all.shape, x_val_all.shape, y_tra.shape, y_val.shape)
    
    for n, column_selection_ds_n in column_selection_ds.items():
        column_selection_ds_n['metrics'] = []
        for name, sel_idxs, desc in zip(column_selection_ds_n['strategy_name'], 
                                        column_selection_ds_n['column_idxs'], 
                                        column_selection_ds_n['strategy_desc']
                                       ):
            x_tra = x_tra_all[:, sel_idxs]
            x_val = x_val_all[:, sel_idxs]

            m = call_back_per_ds(x_tra, y_tra, x_val, y_val, desc, f'{ds}, nf={n}: {name}')
            column_selection_ds_n['metrics'].append(m)

        # PCA part:
        pca = PCA(n_components=n)
        x_tra = pca.fit_transform(x_tra_vn)
        x_val = pca.transform(x_val_vn)

        desc = 'PCA of all features. Columns are not valid'
        name = 'pca'
        m = call_back_per_ds(x_tra, y_tra, x_val, y_val, desc, f'{ds}, nf={n}: {name}')
        column_selection_ds_n['metrics'].append(m)
        column_selection_ds_n['strategy_name'].append(name)
        column_selection_ds_n['column_idxs'].append([])
        column_selection_ds_n['strategy_desc'].append(desc)
    

In [ ]:
save_pckl(column_selection, data_root/'features'/'selected_idx_evaluated.pckl')

In [ ]:
column_selection['actin'][20]['strategy_name']